In [213]:
from numba import njit
from numba.openmp import openmp_context as openmp
from numba.openmp import omp_get_thread_num, omp_get_num_threads, omp_set_num_threads, omp_get_max_threads, omp_get_wtime
import numpy as np
from numba import njit
from numba.openmp import openmp_context as openmp
import random

In [214]:
@njit
def hello():
    with openmp('parallel'):
        thread_id = omp_get_thread_num()
        num_threads = omp_get_num_threads()
        print('hello world t', thread_id, ' of ', num_threads)
        
@njit        
def printNumThreads():
    with openmp('parallel'):
        if (omp_get_thread_num() == 0):
            print("omp_get_max_threads() =", omp_get_max_threads())
            print("omp_get_num_threads() =", omp_get_num_threads())
            
@njit
def test_pi_loop():
    num_steps = 100000
    step = 1.0 / num_steps

    the_sum = 0.0
    #omp_set_num_threads(4)

    with openmp("parallel"):
        with openmp("for reduction(+:the_sum) schedule(static)"):
            for j in range(num_steps):
                c = step
                x = ((j-1) - 0.5) * step
                the_sum += 4.0 / (1.0 + x * x)

    pi = step * the_sum
    return pi

@njit
def setNumThreads(n):
    omp_set_num_threads(n)

@njit
def isArraySorted(arr):
    n = len(arr)
    if (n > 0):
        prev = arr[0]
        for i in range(n):
            if (arr[i] < prev):
                return False
    return True;
    
def getArr(n):
    arr = [0] * n
    for i in range(n):
        arr[i] = random.randint(1, 100)
    return arr
    
def createRandomArr(n):
    return np.random.randint(0, 100, N)

@njit
def getOmpTime():
    return omp_get_wtime()

In [215]:
setNumThreads(4)

In [216]:
setNumThreads(8)

In [217]:
setNumThreads(2)

In [218]:
printNumThreads()
hello()

omp_get_max_threads() = 2
omp_get_num_threads() = 2
hello world t 0  of  2
hello world t 1  of  2


In [219]:
%%time
for i in range(1000):
    test_pi_loop()

CPU times: user 237 ms, sys: 10.1 ms, total: 247 ms
Wall time: 3.04 s


In [220]:
%%time
for i in range(1000):
    test_pi_loop()

CPU times: user 129 ms, sys: 7.5 ms, total: 136 ms
Wall time: 70.5 ms


In [221]:
@njit
def quickSortSeq(arr, low, high):
    if (low < high):
        # partitionHoar
        i = low
        j = high
        pivot = arr[(i + j) // 2]
        while (1):
            while (arr[i] < pivot):
                i = i + 1
            while (arr[j] > pivot):
                j = j - 1
        
            if (i >= j):
                break
        
            arr[i], arr[j] = arr[j], arr[i]
            i = i + 1
            j = j - 1
        pi = j

        # recurcive call
        quickSortSeq(arr, low, pi)
        quickSortSeq(arr, pi + 1, high)

@njit
def quickSortPar(arr, low, high, max_d, d = 0):
    if (low < high):
        # partitionHoar
        i = low
        j = high
        pivot = arr[(i + j) // 2]
        while (1):
            while (arr[i] < pivot):
                i = i + 1
            while (arr[j] > pivot):
                j = j - 1
        
            if (i >= j):
                break
        
            arr[i], arr[j] = arr[j], arr[i]
            i = i + 1
            j = j - 1
        pi = j

        # recurcive call
        if (d < max_d):
            #with openmp("parallel sections"):
            with openmp("task shared(arr, low, pi)"):
                quickSortPar(arr, low, pi, max_d, d + 1)
                #quickSortSeq(arr, low, pi)
            with openmp("task shared(arr, pi, high)"):    
                quickSortPar(arr, pi + 1, high, max_d, d + 1)  
                #quickSortSeq(arr, pi + 1, high)
        else:
            quickSortPar(arr, low, pi, max_d, d + 1)
            quickSortPar(arr, pi + 1, high, max_d, d + 1)
        #quickSortPar(arr, low, pi)
        #quickSortPar(arr, pi + 1, high)

In [222]:
N = 10
ITERS = 1

In [223]:
arr = createRandomArr(N)
print(arr)

[61 68 45 79 91 90 62 91 88 21]


In [224]:
with openmp("parallel"):
    with openmp("single"):
        quickSortPar(arr, 0, len(arr)-1, 1)
print(arr)

[21 45 61 62 68 79 88 90 91 91]


In [225]:
arr = createRandomArr(N)
print(arr)
with openmp("parallel"):
    with openmp("single"):
        quickSortPar(arr, 0, len(arr)-1, 1)
print(arr)

[92 44 81 22 99 98 63  6 48 40]
[ 6 22 40 44 48 63 81 92 98 99]


In [226]:
def printQuickSortPar():
    start_time = getOmpTime()
    arr = createRandomArr(N)
    print(arr)
    with openmp("parallel"):
        with openmp("single"):
            quickSortPar(arr, 0, len(arr)-1, 1)
    print(arr)

printQuickSortPar()

[15 48 59 15 12 26 33 43 92 27]
[12 15 15 26 27 33 43 48 59 92]


In [227]:
arr = createRandomArr(N)
print(arr)

[77 41 18 84 75 28 83 88  0 41]


In [228]:
quickSortSeq(arr, 0, len(arr)-1)
print(arr)

[ 0 18 28 41 41 75 77 83 84 88]


In [229]:
N = 10000
ITERS = 100

In [230]:
def calcSeqTime():
    arr = createRandomArr(10)
    quickSortSeq(arr, 0, len(arr)-1)
    full_time = 0
    for it in range(ITERS):
        #arr = getArr(N)
        arr = createRandomArr(N)
        #print(arr)
        
        start_time = getOmpTime()
    
        quickSortSeq(arr, 0, len(arr)-1)
        
        end_time = getOmpTime()
        elapsed_time = end_time - start_time;
        #print(elapsed_time)
        full_time += elapsed_time
        
        #print(arr)
        if (not isArraySorted(arr)):
            print("Massive is not sorted\n")
        
    full_time /= ITERS
    print("Time taken (sequential):", full_time, "seconds")

In [231]:
for i in range(10):
    calcSeqTime()

Time taken (sequential): 0.00039597272872924803 seconds
Time taken (sequential): 0.0003652071952819824 seconds
Time taken (sequential): 0.0004100584983825684 seconds
Time taken (sequential): 0.00041363716125488283 seconds
Time taken (sequential): 0.0003786182403564453 seconds
Time taken (sequential): 0.0003703737258911133 seconds
Time taken (sequential): 0.000368037223815918 seconds
Time taken (sequential): 0.0003619265556335449 seconds
Time taken (sequential): 0.0003656768798828125 seconds
Time taken (sequential): 0.000382390022277832 seconds


In [235]:
def calcParTime():
    arr = createRandomArr(10)
    with openmp("parallel"):
            with openmp("single"):
                quickSortPar(arr, 0, len(arr)-1, 3)
    full_time = 0
    for it in range(ITERS):
        #arr = getArr(N)
        arr = createRandomArr(N)
        #print(arr)
        
        start_time = getOmpTime()
        
        with openmp("parallel"):
            with openmp("single"):
                quickSortPar(arr, 0, len(arr)-1, 3)
        #quickSortPar(arr, 0, len(arr)-1)
        
        end_time = getOmpTime()
        elapsed_time = end_time - start_time;
        #print(elapsed_time)
        full_time += elapsed_time
        
        #print(arr)
        
        if (not isArraySorted(arr)):
            print("Massive is not sorted\n")
        
    full_time /= ITERS
    print(f"Time taken ({2} threads):", full_time, "seconds")
    return full_time

In [236]:
for i in range(10):
    calcParTime()

Time taken (2 threads): 0.0006250143051147461 seconds
Time taken (2 threads): 0.00044263362884521484 seconds
Time taken (2 threads): 0.00044116973876953127 seconds
Time taken (2 threads): 0.0004506635665893555 seconds
Time taken (2 threads): 0.0004828906059265137 seconds
Time taken (2 threads): 0.0004530882835388184 seconds
Time taken (2 threads): 0.000433499813079834 seconds
Time taken (2 threads): 0.0004293656349182129 seconds
Time taken (2 threads): 0.00043916940689086914 seconds
Time taken (2 threads): 0.0004310011863708496 seconds


In [ ]:
printNumThreads()
t = calcParTime()
while (t > 0.00036):
    t = calcParTime()

# TESTING

In [ ]:
@njit
def my_print(val):
    print(val)
    
@njit
def getArr(n):
    arr = [0] * n
    for i in range(n):
        arr[i] = random.randint(1, 100)
    return arr

@njit
def my_test():
    arr = getArr(N)
    my_print(arr)
    with openmp("parallel shared(arr)"):
        with openmp("single shared(arr)"):
            quickSortPar(arr, 0, len(arr)-1)
    my_print(arr)
            
my_test()

# OLD

In [27]:
import timeit
import random

def quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quicksort(left) + middle + quicksort(right)

# Генерация случайного массива
random_array = [random.randint(1, 1000) for _ in range(10000)]

# Замер времени выполнения
num_iterations = 100
for i in range (3):
    total_time = timeit.timeit(lambda: quicksort(random_array), number=num_iterations)

    # Усреднение времени
    average_time = total_time / num_iterations
    print(f"Среднее время выполнения: {average_time} секунд")


Среднее время выполнения: 0.010098171999998158 секунд
Среднее время выполнения: 0.009548747999997432 секунд
Среднее время выполнения: 0.00960765399999218 секунд


In [104]:
random_array = [random.randint(1, 1000) for _ in range(10)]
print(random_array)
sorted_array = quicksort(random_array)
print(sorted_array)

[53, 122, 295, 976, 317, 35, 854, 760, 106, 721]
[35, 53, 106, 122, 295, 317, 721, 760, 854, 976]


In [29]:
def calcSeqTime2():
    full_time = 0
    for it in range(ITERS):
        arr = [0] * N
        for i in range(N):
            arr[i] = random.randint(1, 100)
        #print(arr)
        
        start_time = getOmpTime()
    
        arr = quicksort(arr)
        
        end_time = getOmpTime()
        elapsed_time = end_time - start_time;
        #print(elapsed_time)
        full_time += elapsed_time
        
        #print(arr)
        if (not isArraySorted(arr)):
            print("Massive is not sorted\n")
        
    full_time /= ITERS
    print("Time taken (sequential):", full_time, "seconds")

calcSeqTime2()

Time taken (sequential): 0.005700380802154541 seconds
